In [ ]:
import copy
import itertools as it
import logging
import random
import sys
import typing

from hstrat import _auxiliary_lib as hstrat_aux
import interval_search as inch
from IPython.core.display import display, HTML
from matplotlib import pyplot as plt
from nbmetalog import nbmetalog as nbm
import numpy as np
import opytional as opyt
import pandas as pd
import pytest
from scipy import stats as scipy_stats
import seaborn as sns
from tqdm import tqdm

import pylib

random.seed(1)

logging.basicConfig(
    format="[%(funcName)s:%(lineno)d] %(message)s",
)
logger = logging.getLogger()

pylib.jupyter_hide_toggle(hide=True)


In [ ]:
nbm.print_metadata()

pylib.jupyter_hide_toggle(hide=True)


# Section 1: Implement `get_deposition_site_at_rank`


## Section 1a: Prepare Support


In [ ]:
def get_num_sites_reserved_per_incidence_at_rank(rank: int) -> int:
    return pylib.bit_ceil(
        pylib.hanoi.get_max_hanoi_value_through_index(rank) + 1
    )


In [ ]:
# test get_num_sites_reserved_per_incidence_at_rank
assert [
    get_num_sites_reserved_per_incidence_at_rank(rank) for rank in range(17)
] == [
    # hanoi sequence (1-based):
    1,  # 1,
    2,  # 2,
    2,  # 1,
    4,  # 3,
    4,  # 1,
    4,  # 2,
    4,  # 1,
    4,  # 4,
    4,  # 1,
    4,  # 2,
    4,  # 1,
    4,  # 3,
    4,  # 1,
    4,  # 2,
    4,  # 1,
    8,  # 5,
    8,  # 1,
]
pylib.jupyter_hide_toggle(hide=True)


In [ ]:
@pylib.log_args_and_result(logger, logging.DEBUG)
def get_num_incidence_reservations_at_rank(
    rank: int, surface_size: int
) -> int:
    reservation_size = get_num_sites_reserved_per_incidence_at_rank(rank)
    assert surface_size % reservation_size == 0
    num_reservations = surface_size // reservation_size
    return num_reservations


In [ ]:
# test get_num_incidence_reservations_at_rank
assert [
    get_num_incidence_reservations_at_rank(rank, 64) for rank in range(17)
] == [
    # hanoi sequence (1-based):
    64,  # 1,
    32,  # 2,
    32,  # 1,
    16,  # 3,
    16,  # 1,
    16,  # 2,
    16,  # 1,
    16,  # 4,
    16,  # 1,
    16,  # 2,
    16,  # 1,
    16,  # 3,
    16,  # 1,
    16,  # 2,
    16,  # 1,
    8,   # 5,
    8,   # 1,
]

pylib.jupyter_hide_toggle(hide=True)


In [ ]:
def get_surface_rank_capacity(surface_size: int) -> int:
    return pylib.hanoi.get_index_of_hanoi_value_nth_incidence(surface_size, 0)


In [ ]:
for surface_size_exp in range(2, 12):
    surface_size = 2**surface_size_exp
    get_num_incidence_reservations_at_rank(
        get_surface_rank_capacity(surface_size) - 1,
        surface_size,
    )
    with pytest.raises(AssertionError):
        get_num_incidence_reservations_at_rank(
            get_surface_rank_capacity(surface_size),
            surface_size,
        )

pylib.jupyter_hide_toggle(hide=True)


In [ ]:
def is_2x_reservation_eligible(
    hanoi_value: int, surface_size: int, rank: int
) -> bool:
    reservation_width = get_num_sites_reserved_per_incidence_at_rank(rank)
    lb_inclusive = (
        pylib.hanoi.get_max_hanoi_value_through_index(rank)
        - reservation_width // 2
        + 1
    )
    ub_exclusive = reservation_width // 2
    return lb_inclusive <= hanoi_value < ub_exclusive


pylib.jupyter_hide_toggle()


In [ ]:
# test is_2x_reservation_eligible
# TODO

pylib.jupyter_hide_toggle(hide=True)


In [ ]:
def get_prev_invading_hanoi_value(
    hanoi_value: int, rank: int
) -> typing.Optional[int]:
    reservation_width = get_num_sites_reserved_per_incidence_at_rank(rank)
    degrading_reservation_width = reservation_width // 2

    if hanoi_value < degrading_reservation_width // 2:
        return hanoi_value + degrading_reservation_width // 2
    else:
        return None


In [ ]:
assert (
    get_prev_invading_hanoi_value(
        1, pylib.hanoi.get_index_of_hanoi_value_nth_incidence(2, 0)
    )
    is None
)

assert (
    get_prev_invading_hanoi_value(
        3, pylib.hanoi.get_index_of_hanoi_value_nth_incidence(2, 0)
    )
    is None
)

assert (
    get_prev_invading_hanoi_value(
        3, pylib.hanoi.get_index_of_hanoi_value_nth_incidence(4, 0)
    )
    is None
)

assert (
    get_prev_invading_hanoi_value(
        1, pylib.hanoi.get_index_of_hanoi_value_nth_incidence(4, 0)
    )
    == 3
)

pylib.jupyter_hide_toggle(hide=True)


In [ ]:
def is_hanoi_invader(hanoi_value: int, rank: int) -> bool:
    reservation_width = get_num_sites_reserved_per_incidence_at_rank(rank)
    return hanoi_value >= reservation_width // 2


In [ ]:
assert not is_hanoi_invader(
    1, pylib.hanoi.get_index_of_hanoi_value_nth_incidence(2, 0)
)
assert is_hanoi_invader(
    3, pylib.hanoi.get_index_of_hanoi_value_nth_incidence(2, 0)
)

assert not is_hanoi_invader(
    3, pylib.hanoi.get_index_of_hanoi_value_nth_incidence(4, 0)
)

assert not is_hanoi_invader(
    1, pylib.hanoi.get_index_of_hanoi_value_nth_incidence(4, 0)
)

pylib.jupyter_hide_toggle(hide=True)


In [ ]:
def is_hanoi_invaded(hanoi_value: int, rank: int) -> bool:
    reservation_width = get_num_sites_reserved_per_incidence_at_rank(rank)
    max_hanoi_value = pylib.hanoi.get_max_hanoi_value_through_index(rank)

    return (
        not is_hanoi_invader(hanoi_value, rank)
        and max_hanoi_value >= hanoi_value + reservation_width // 2
    )


In [ ]:
assert not is_hanoi_invaded(
    1, pylib.hanoi.get_index_of_hanoi_value_nth_incidence(2, 0)
)
assert is_hanoi_invaded(
    0, pylib.hanoi.get_index_of_hanoi_value_nth_incidence(2, 0)
)
assert not is_hanoi_invaded(
    3, pylib.hanoi.get_index_of_hanoi_value_nth_incidence(2, 0)
)

assert not is_hanoi_invaded(
    3, pylib.hanoi.get_index_of_hanoi_value_nth_incidence(4, 0)
)
assert is_hanoi_invaded(
    0, pylib.hanoi.get_index_of_hanoi_value_nth_incidence(4, 0)
)
assert not is_hanoi_invaded(
    1, pylib.hanoi.get_index_of_hanoi_value_nth_incidence(4, 0)
)
assert is_hanoi_invaded(
    1, pylib.hanoi.get_index_of_hanoi_value_nth_incidence(5, 0)
)

pylib.jupyter_hide_toggle(hide=True)


In [ ]:
def is_hanoi_invadable_and_uninvaded(hanoi_value: int, rank: int) -> bool:
    return not is_hanoi_invader(hanoi_value, rank) and not is_hanoi_invaded(
        hanoi_value, rank
    )


In [ ]:
assert is_hanoi_invadable_and_uninvaded(
    1, pylib.hanoi.get_index_of_hanoi_value_nth_incidence(2, 0)
)
assert not is_hanoi_invadable_and_uninvaded(
    2, pylib.hanoi.get_index_of_hanoi_value_nth_incidence(2, 0)
)
assert not is_hanoi_invadable_and_uninvaded(
    0, pylib.hanoi.get_index_of_hanoi_value_nth_incidence(2, 0)
)
assert not is_hanoi_invadable_and_uninvaded(
    3, pylib.hanoi.get_index_of_hanoi_value_nth_incidence(2, 0)
)
assert is_hanoi_invadable_and_uninvaded(
    3, pylib.hanoi.get_index_of_hanoi_value_nth_incidence(4, 0)
)
assert not is_hanoi_invadable_and_uninvaded(
    0, pylib.hanoi.get_index_of_hanoi_value_nth_incidence(4, 0)
)
assert is_hanoi_invadable_and_uninvaded(
    1, pylib.hanoi.get_index_of_hanoi_value_nth_incidence(4, 0)
)
assert not is_hanoi_invadable_and_uninvaded(
    1, pylib.hanoi.get_index_of_hanoi_value_nth_incidence(5, 0)
)
assert not is_hanoi_invadable_and_uninvaded(
    5, pylib.hanoi.get_index_of_hanoi_value_nth_incidence(5, 0)
)
pylib.jupyter_hide_toggle(hide=True)


In [ ]:
def iter_hanoi_invader_values(hanoi_value: int) -> typing.Iterator[int]:
    for i in it.count():
        yield hanoi_value + pylib.bit_ceil(hanoi_value + 1) * 2**i


In [ ]:
assert [*it.islice(iter_hanoi_invader_values(0), 6)] == [1, 2, 4, 8, 16, 32]
assert [*it.islice(iter_hanoi_invader_values(3), 4)] == [7, 11, 19, 35]
assert [*it.islice(iter_hanoi_invader_values(8), 2)] == [24, 40]

pylib.jupyter_hide_toggle(hide=True)


In [ ]:
def has_hanoi_value_filled_first_reservation_layer(
    hanoi_value: int, surface_size: int, rank: int
):
    first_instance_index = pylib.hanoi.get_index_of_hanoi_value_nth_incidence(
        hanoi_value, 0
    )
    if first_instance_index >= get_surface_rank_capacity(surface_size):
        return False

    first_layer_size = get_num_incidence_reservations_at_rank(
        pylib.hanoi.get_index_of_hanoi_value_nth_incidence(hanoi_value, 0),
        surface_size,
    )
    return not (
        pylib.hanoi.get_incidence_count_of_hanoi_value_through_index(
            hanoi_value,
            rank,
        )
        < first_layer_size
    )


In [ ]:
def get_upcoming_hanoi_invasion_value(hanoi_value: int, rank: int) -> int:
    rank = max(
        rank,
        pylib.hanoi.get_index_of_hanoi_value_nth_incidence(hanoi_value, 0),
    )

    reservation_width = get_num_sites_reserved_per_incidence_at_rank(rank)
    if is_hanoi_invader(hanoi_value, rank) or is_hanoi_invaded(
        hanoi_value, rank
    ):
        return hanoi_value + reservation_width
    else:
        return hanoi_value + reservation_width // 2


In [ ]:
def get_upcoming_hanoi_invasion_rank(hanoi_value: int, rank: int) -> int:
    upcoming_invasion_hanoi_value = get_upcoming_hanoi_invasion_value(
        hanoi_value, rank
    )
    upcoming_invasion_rank = (
        pylib.hanoi.get_index_of_hanoi_value_nth_incidence(
            upcoming_invasion_hanoi_value, 0
        )
    )
    return upcoming_invasion_rank


In [ ]:
def get_regime_end_rank(hanoi_value: int, rank: int) -> int:
    return get_upcoming_hanoi_invasion_rank(hanoi_value, rank)


In [ ]:
def get_regime_mx(hanoi_value: int, rank: int) -> int:
    if is_hanoi_invadable_and_uninvaded(hanoi_value, rank):
        return 2
    else:
        return 1


In [ ]:
def get_regime_num_reservations_available(
    hanoi_value: int, surface_size: int, rank: int
) -> int:
    mx = get_regime_mx(hanoi_value, rank)
    return get_num_incidence_reservations_at_rank(rank, surface_size) * mx


In [ ]:
def get_safe_downgrade_rank(
    cycle_num_ranks: int,
    required_cycle_rank_position: int,
    required_lag: int,
    end_rank: int,
    hanoi_offset: int,
    cadence_for_asserts: int,
    hanoi_value_for_asserts: int,
):

    offset = hanoi_offset
    hanoi_value = hanoi_value_for_asserts

    assert cadence_for_asserts
    cadence = cadence_for_asserts

    assert cycle_num_ranks
    assert required_cycle_rank_position < cycle_num_ranks
    assert required_lag % cadence == 0
    # oc is offset corrected
    end_rank_oc = end_rank - offset

    deadline_rank_oc = end_rank_oc - required_lag
    #
    #    tt_below |             | goal
    #########################################################
    #             |----tour time----|
    #             |             :   |
    #                                           ^ end_rank
    #                              ^deadline rank
    #               ^mapped to here (intermediate)
    #             ^ then to here
    #                            ^ then to here
    #
    #########################################################
    #             |----tour time----|
    #             |             :   |          :  |
    #                                                    ^ end_rank
    #                                         ^deadline rank
    #                              ^mapped to here (intermediate)
    #             ^ then to here
    #                           ^ then to here
    #
    intermediate_oc = deadline_rank_oc - required_cycle_rank_position
    assert deadline_rank_oc - intermediate_oc <= cycle_num_ranks

    tt_below_oc = intermediate_oc - pylib.modulo(
        intermediate_oc, cycle_num_ranks
    )
    assert tt_below_oc % cadence == 0
    assert tt_below_oc % cycle_num_ranks == 0
    assert (
        deadline_rank_oc - tt_below_oc
        <= cycle_num_ranks + required_cycle_rank_position
    )

    goal_oc = tt_below_oc + required_cycle_rank_position
    assert goal_oc % cycle_num_ranks == required_cycle_rank_position
    assert goal_oc % cadence == 0
    assert goal_oc <= deadline_rank_oc
    assert goal_oc - deadline_rank_oc <= cycle_num_ranks

    downgrade_rank_oc = goal_oc
    downgrade_rank = goal_oc + offset
    assert downgrade_rank <= end_rank
    assert (
        required_lag
        <= end_rank - downgrade_rank
        <= cycle_num_ranks + required_lag
    )

    assert cycle_num_ranks % cadence == 0
    assert (
        pylib.hanoi.get_incidence_count_of_hanoi_value_through_index(
            hanoi_value, offset
        )
    ) % cycle_num_ranks == 1
    assert (
        pylib.hanoi.get_incidence_count_of_hanoi_value_through_index(
            hanoi_value, offset + cadence
        )
    ) % cycle_num_ranks == 2

    assert cycle_num_ranks % cadence == 0
    if tt_below_oc + offset >= 0:
        assert tt_below_oc % cadence == 0
        assert tt_below_oc % cycle_num_ranks == 0
        assert (
            pylib.hanoi.get_incidence_count_of_hanoi_value_through_index(
                hanoi_value, tt_below_oc + offset
            )
            - 1
        ) % (cycle_num_ranks // cadence) == 0
    if downgrade_rank >= 0:
        assert (downgrade_rank + cadence - offset) % cadence == 0
        assert (
            pylib.hanoi.get_incidence_count_of_hanoi_value_through_index(
                hanoi_value, downgrade_rank
            )
            - 1
        ) % (cycle_num_ranks // cadence) == (
            required_cycle_rank_position // cadence
        )

    return downgrade_rank


In [ ]:
def get_regime_reservation_downgrade_rank(
    hanoi_value: int, surface_size: int, rank: int
) -> int:
    cadence = pylib.hanoi.get_hanoi_value_index_cadence(hanoi_value)
    offset = pylib.hanoi.get_hanoi_value_index_offset(hanoi_value)
    big_tour_size = get_regime_num_reservations_available(
        hanoi_value,
        surface_size,
        rank,
    )
    if big_tour_size == 1:
        return sys.maxsize
    big_tour_time = big_tour_size * cadence
    end_rank = get_regime_end_rank(hanoi_value, rank)
    assert end_rank

    cycle_num_ranks = big_tour_time
    required_cycle_rank_position = big_tour_time // 2 - cadence
    assert required_cycle_rank_position % cadence == 0
    assert required_cycle_rank_position < cycle_num_ranks
    required_lag = big_tour_time // 2
    assert required_lag % cadence == 0

    downgrade_rank = get_safe_downgrade_rank(
        cycle_num_ranks=cycle_num_ranks,
        required_cycle_rank_position=required_cycle_rank_position,
        required_lag=required_lag,
        end_rank=end_rank,
        hanoi_offset=offset,
        cadence_for_asserts=cadence,
        hanoi_value_for_asserts=hanoi_value,
    )
    assert downgrade_rank <= end_rank
    return downgrade_rank


In [ ]:
def get_fractional_downgrade_state(
    hanoi_value: int, surface_size: int, rank: int
) -> typing.Dict:
    for invading_hanoi_value in iter_hanoi_invader_values(hanoi_value):
        if invading_hanoi_value >= 2**surface_size // 2:
            return None
        if not has_hanoi_value_filled_first_reservation_layer(
            invading_hanoi_value, surface_size, rank
        ):
            break

    assert invading_hanoi_value > hanoi_value

    for attempt in 1, 2:
        invader_cadence = pylib.hanoi.get_hanoi_value_index_cadence(
            invading_hanoi_value
        )
        protagonist_cadence = pylib.hanoi.get_hanoi_value_index_cadence(
            hanoi_value
        )

        if invading_hanoi_value > surface_size // 2:
            return None

        invasion_rank = pylib.hanoi.get_index_of_hanoi_value_nth_incidence(
            invading_hanoi_value, 0
        )
        num_reservations = get_num_incidence_reservations_at_rank(
            invasion_rank, surface_size
        )
        if num_reservations == 1:
            return None

        tour_size = 2 * num_reservations

        protagonist_tour_time = tour_size * protagonist_cadence
        num_protagonist_cycles_per_invader_cadence = (
            invader_cadence // protagonist_tour_time
        )
        if num_protagonist_cycles_per_invader_cadence >= (
            # see test_calc_dyadic_lcm_upper_bound.py
            # https://github.com/mmore500/hstrat-surface-concept/blob/4d0eca2ae3fe1890983889fdb8a8482fd3b24627/pylib/test/test_calc_dyadic_lcm_upper_bound.py#L39
            4
            * tour_size
        ):
            break

        if attempt == 1 and invasion_rank <= rank:
            invading_hanoi_value = (
                hanoi_value + (invading_hanoi_value - hanoi_value) * 2
            )
    else:
        return None

    # see test_calc_dyadic_lcm_upper_bound.py
    # https://github.com/mmore500/hstrat-surface-concept/blob/4d0eca2ae3fe1890983889fdb8a8482fd3b24627/pylib/test/test_calc_dyadic_lcm_upper_bound.py#L39
    # num_granules * tour_size * 2 == num_protagonist_cycles_per_invader_cadence
    # num_granules == num_protagonist_cycles_per_invader_cadence // (tour_size * 2)
    num_granules = min(
        num_protagonist_cycles_per_invader_cadence // (tour_size * 2),
        num_reservations,
    )
    assert 1 < num_granules <= num_reservations, (
        num_granules,
        num_reservations,
    )
    granule_size = num_reservations // num_granules
    assert granule_size

    raw_current_subtrahend = (
        pylib.hanoi.get_incidence_count_of_hanoi_value_through_index(
            invading_hanoi_value,
            rank,
        )
    )
    raw_next_subtrahend = raw_current_subtrahend + 1

    current_subtrahend_granule = (
        raw_current_subtrahend + granule_size - 1
    ) // granule_size
    assert current_subtrahend_granule <= raw_current_subtrahend
    assert bool(current_subtrahend_granule) == bool(raw_current_subtrahend)

    next_subtrahend_granule = (
        raw_next_subtrahend + granule_size - 1
    ) // granule_size
    assert next_subtrahend_granule <= raw_next_subtrahend
    assert bool(next_subtrahend_granule) == bool(raw_next_subtrahend)

    granularized_current_subtrahend = current_subtrahend_granule * granule_size
    assert granularized_current_subtrahend >= raw_current_subtrahend

    granularized_next_subtrahend = next_subtrahend_granule * granule_size
    assert granularized_next_subtrahend >= raw_next_subtrahend

    assert (
        0
        <= granularized_current_subtrahend
        <= granularized_next_subtrahend
        <= num_reservations
    )

    upcoming_invader_rank = (
        pylib.hanoi.get_index_of_hanoi_value_next_incidence(
            invading_hanoi_value, rank
        )
    )
    assert upcoming_invader_rank > rank
    return {
        "hanoi value": hanoi_value,
        "hanoi cadence": protagonist_cadence,
        "upcoming invader rank": upcoming_invader_rank,
        "first invasion rank": invasion_rank,
        "invader cadence": invader_cadence,
        "invading hanoi value": invading_hanoi_value,
        "current subtrahend": granularized_current_subtrahend,
        "next subtrahend": granularized_next_subtrahend,
        "tour size": tour_size,
        "num granules": num_granules,
        "granule size": granule_size,
    }


In [ ]:
@pylib.log_args_and_result(logger, logging.DEBUG)
def get_fractional_downgrade_rank(
    hanoi_value: int,
    surface_size: int,
    rank: int,
    fractional_downgrade_state: typing.Dict,
) -> int:
    state = fractional_downgrade_state

    cadence = pylib.hanoi.get_hanoi_value_index_cadence(hanoi_value)
    offset = pylib.hanoi.get_hanoi_value_index_offset(hanoi_value)
    big_tour_size = pylib.calc_dyadic_lcm_upper_bound(
        state["tour size"] - state["current subtrahend"],
        state["tour size"] - state["next subtrahend"],
    )

    if state["tour size"] == 1:
        return sys.maxsize
    big_tour_time = big_tour_size * cadence
    end_rank = state["upcoming invader rank"]
    assert end_rank

    cycle_num_ranks = big_tour_time
    required_cycle_rank_position = 0

    assert required_cycle_rank_position % cadence == 0
    assert required_cycle_rank_position < cycle_num_ranks
    required_lag = (state["tour size"] - state["next subtrahend"]) * cadence
    assert required_lag % cadence == 0
    res = get_safe_downgrade_rank(
        cycle_num_ranks=cycle_num_ranks,
        required_cycle_rank_position=required_cycle_rank_position,
        required_lag=required_lag,
        end_rank=end_rank,
        hanoi_offset=offset,
        cadence_for_asserts=cadence,
        hanoi_value_for_asserts=hanoi_value,
    )

    assert res > state["upcoming invader rank"] - state["invader cadence"]
    assert res <= state["upcoming invader rank"]
    assert res > 0
    return res


In [ ]:
@pylib.log_args_and_result(logger, logging.DEBUG)
def get_fractional_downgrade_num_reservations_provided(
    hanoi_value: int,
    surface_size: int,
    rank: int,
    fractional_downgrade_state: typing.Dict,
) -> int:
    thresh = get_fractional_downgrade_rank(
        hanoi_value,
        surface_size,
        rank,
        fractional_downgrade_state,
    )

    state = fractional_downgrade_state
    if rank >= thresh:
        return state["tour size"] - state["next subtrahend"]
    else:
        return state["tour size"] - state["current subtrahend"]


In [ ]:
@pylib.log_args_and_result(logger, logging.DEBUG)
def get_regime_num_reservations_provided(
    hanoi_value: int, surface_size: int, rank: int
) -> int:
    thresh = get_regime_reservation_downgrade_rank(
        hanoi_value, surface_size, rank
    )
    before_thresh_num = get_regime_num_reservations_available(
        hanoi_value,
        surface_size,
        rank,
    )
    if rank >= thresh:
        return before_thresh_num // 2
    else:
        return before_thresh_num


In [ ]:
@pylib.log_args_and_result(logger, logging.DEBUG)
def get_num_reservations_provided(
    hanoi_value: int, surface_size: int, rank: int
) -> int:
    fractional_downgrade_state = get_fractional_downgrade_state(
        hanoi_value, surface_size, rank
    )
    if fractional_downgrade_state is not None:
        return get_fractional_downgrade_num_reservations_provided(
            hanoi_value,
            surface_size,
            rank,
            fractional_downgrade_state,
        )
    else:
        return get_regime_num_reservations_provided(
            hanoi_value, surface_size, rank
        )


## Section 1b: Define `get_deposition_site_at_rank`


In [ ]:
def get_deposition_site_at_rank(rank: int, surface_size: int) -> int:
    if rank > get_surface_rank_capacity(surface_size):
        raise ValueError(
            f"{surface_size}-bit surface only valid "
            "through rank <= {get_surface_rank_capacity(surface_size)}, "
            f"rank {rank} was requested"
        )

    within_reservation_offset = pylib.hanoi.get_hanoi_value_at_index(rank)

    num_incidence_reservations = get_num_reservations_provided(
        within_reservation_offset, surface_size, rank
    )

    reservation_index = (
        pylib.hanoi.get_hanoi_value_incidence_at_index(rank)
        % num_incidence_reservations
    )

    longevity_ordered_reservation_position = (
        # this is more correct in a theoretical sense, and does have some effect
        # but actual functional improvements relative to simpler naive/alternating
        # should be tested; it may not be worth the extra complexity
        pylib.longevity_ordering_descending.get_longevity_mapped_position_of_index(
            reservation_index,
            surface_size,
        )
    )
    if reservation_index == 0:
        assert longevity_ordered_reservation_position == 0
    elif reservation_index == pylib.bit_ceil(num_incidence_reservations) - 1:
        assert longevity_ordered_reservation_position == (
            surface_size // pylib.bit_ceil(num_incidence_reservations)
        )

    res = (
        longevity_ordered_reservation_position
        + within_reservation_offset
    )
    if rank == 0:
        assert res == 0, {
            "longevity_ordered_reservation_position": longevity_ordered_reservation_position,
            "reservation_index": reservation_index,
            "surface_size": surface_size,
            "within_reservation_offset": within_reservation_offset,
        }
    assert res < surface_size
    return res


# Section 2: evaluate `get_deposition_site_at_rank`


## Section 2a: Unit Test


In [ ]:
# test get_deposition_site_at_rank
# **needs to be updated**
# assert [get_deposition_site_at_rank(rank, 64) for rank in range(23)] == [
#          # num_reservations
#          #     # hanoi sequence (0-based):
#     0,   # 64, # 0,
#     1,   # 32, # 1,
#     32,  # 32, # 0,
#     2,   # 16, # 2,
#     16,  # 16, # 0,
#     33,  # 16, # 1,
#     48,  # 16, # 0,
#     3,   # 16, # 3,
#     8,   # 16, # 0,
#     17,  # 16, # 1,
#     24,  # 16, # 0,
#     34,  # 16, # 2,
#     40,  # 16, # 0,
#     49,  # 16, # 1,
#     56,  # 16, # 0,
#     4,   # 8,  # 4,
#     0,   # 8,  # 0,
#     9,   # 8,  # 1,
#     32,  # 8,  # 0,
#     18,  # 8,  # 2,
#     16,  # 8,  # 0,
#     25,  # 8,  # 1,
#     48,  # 8,  # 0,
# ]

# assert [get_deposition_site_at_rank(rank, 16) for rank in range(21)] == [
#         # num_reservations
#         #     # hanoi sequence (0-based):
#     0,  # 16, # 0,
#     1,  # 8,  # 1,
#     8,  # 8,  # 0,
#     2,  # 4,  # 2,
#     4,  # 4,  # 0,
#     9,  # 4,  # 1,
#     12, # 4,  # 0,
#     3,  # 4,  # 3,
#     0,  # 4,  # 0,
#     5,  # 4,  # 1,
#     8,  # 4,  # 0,
#     10, # 4,  # 2,
#     4,  # 4,  # 0,
#     13, # 4,  # 1,
#     12, # 4,  # 0,
#     4,  # 2,  # 4,
#     0,  # 2,  # 0,
#     1,  # 2,  # 1,
#     8,  # 2,  # 0,
#     6,  # 2,  # 2,
#     0,  # 2,  # 0,
# ]

pylib.jupyter_hide_toggle(hide=True)


## Section 2a: Integration Test


## Section 2b: Visualize


# Section 3: Implement `get_deposition_site_at_rank`


## Section 3a: Prepare Support


In [ ]:
# def is_site_reservation_holdover_at_rank(
#     site: int, surface_size: int, rank: int
# ) -> bool:
#     reservation_size = get_num_sites_reserved_per_incidence_at_rank(rank)
#     within_reservation_offset = site % reservation_size

#     return (
#         within_reservation_offset
#         > pylib.hanoi.get_max_hanoi_value_through_index(rank)
#     )


In [ ]:
# # test get_hanoi_value_of_site

# # rank
# #    # deposition site
# #    #     # num_reservations
# #    #     #     # hanoi sequence (0-based):
# # 0  # 0,  # 16, # 0,
# for rank in range(3):
#     assert [
#         is_site_reservation_holdover_at_rank(site, 16, rank)
#         for site in range(16)
#     ] == [False] * 16

# # rank
# #    # deposition site
# #    #     # num_reservations
# #    #     #     # hanoi sequence (0-based):
# # 3  # 2,  # 4,  # 2,
# # 4  # 4,  # 4,  # 0,
# # 5  # 9,  # 4,  # 1,
# # 6  # 12, # 4,  # 0,
# for rank in range(3, 7):
#     assert [
#         is_site_reservation_holdover_at_rank(site, 16, rank)
#         for site in range(16)
#     ] == [
#         False,
#         False,
#         False,
#         True,
#     ] * 4

# # rank
# #    # deposition site
# #    #     # num_reservations
# #    #     #     # hanoi sequence (0-based):
# # 7  # 3,  # 4,  # 3,
# # 8  # 0,  # 4,  # 0,
# # 9  # 5,  # 4,  # 1,
# # 10 # 8,  # 4,  # 0,
# # 11 # 10, # 4,  # 2,
# # 12 # 4,  # 4,  # 0,
# # 13 # 13, # 4,  # 1,
# # 14 # 12, # 4,  # 0,
# for rank in range(7, 14):
#     assert [
#         is_site_reservation_holdover_at_rank(site, 16, rank)
#         for site in range(16)
#     ] == [False] * 16

# # rank
# #    # deposition site
# #    #     # num_reservations
# #    #     #     # hanoi sequence (0-based):
# # 15 # 4,  # 2,  # 4,
# # 16 # 0,  # 2,  # 0,
# # 17 # 1,  # 2,  # 1,
# # 18 # 8,  # 2,  # 0,
# # 19 # 6,  # 2,  # 2,
# # 20 # 0,  # 2,  # 0,
# for rank in range(15, 21):
#     assert [
#         is_site_reservation_holdover_at_rank(site, 16, rank)
#         for site in range(16)
#     ] == [
#         False,
#         False,
#         False,
#         False,
#         False,
#         True,
#         True,
#         True,
#     ] * 2

pylib.jupyter_hide_toggle(hide=True)


In [ ]:
# def get_hanoi_value_of_site_at_rank(
#     site: int, surface_size: int, rank: int
# ) -> int:
#     is_reservation_holdover = is_site_reservation_holdover_at_rank(
#         site, surface_size, rank
#     )

#     reservation_size = get_num_sites_reserved_per_incidence_at_rank(rank)
#     if is_reservation_holdover:
#         reservation_size //= 2

#     return site % reservation_size


pylib.jupyter_hide_toggle(hide=True)


In [ ]:
# # test get_hanoi_value_of_site

# # rank
# #    # deposition site
# #    #     # num_reservations
# #    #     #     # hanoi sequence (0-based):
# # 0  # 0,  # 16, # 0,
# assert [
#     get_hanoi_value_of_site_at_rank(site, 16, 0) for site in range(16)
# ] == [0] * 16

# # rank
# #    # deposition site
# #    #     # num_reservations
# #    #     #     # hanoi sequence (0-based):
# # 1  # 1,  # 8,  # 1,
# # 2  # 8,  # 8,  # 0,
# for rank in 1, 2:
#     assert [
#         get_hanoi_value_of_site_at_rank(site, 16, rank) for site in range(16)
#     ] == [
#         0,
#         1,
#     ] * 8

# # rank
# #    # deposition site
# #    #     # num_reservations
# #    #     #     # hanoi sequence (0-based):
# # 3  # 2,  # 4,  # 2,
# # 4  # 4,  # 4,  # 0,
# # 5  # 9,  # 4,  # 1,
# # 6  # 12, # 4,  # 0,
# for rank in range(3, 7):
#     assert [
#         get_hanoi_value_of_site_at_rank(site, 16, rank) for site in range(16)
#     ] == [
#         0,
#         1,
#         2,
#         1,
#     ] * 4

# # rank
# #    # deposition site
# #    #     # num_reservations
# #    #     #     # hanoi sequence (0-based):
# # 7  # 3,  # 4,  # 3,
# # 8  # 0,  # 4,  # 0,
# # 9  # 5,  # 4,  # 1,
# # 10 # 8,  # 4,  # 0,
# # 11 # 10, # 4,  # 2,
# # 12 # 4,  # 4,  # 0,
# # 13 # 13, # 4,  # 1,
# # 14 # 12, # 4,  # 0,
# for rank in range(7, 14):
#     assert [
#         get_hanoi_value_of_site_at_rank(site, 16, rank) for site in range(16)
#     ] == [
#         0,
#         1,
#         2,
#         3,
#     ] * 4

# # rank
# #    # deposition site
# #    #     # num_reservations
# #    #     #     # hanoi sequence (0-based):
# # 15 # 4,  # 2,  # 4,
# # 16 # 0,  # 2,  # 0,
# # 17 # 1,  # 2,  # 1,
# # 18 # 8,  # 2,  # 0,
# # 19 # 6,  # 2,  # 2,
# # 20 # 0,  # 2,  # 0,
# for rank in range(15, 21):
#     assert [
#         get_hanoi_value_of_site_at_rank(site, 16, rank) for site in range(16)
#     ] == [
#         0,
#         1,
#         2,
#         3,
#         4,
#         1,
#         2,
#         3,
#     ] * 2

pylib.jupyter_hide_toggle(hide=True)


In [ ]:
# def get_reservation_index_of_site_at_rank(
#     site: int, surface_size: int, rank: int
# ) -> int:
#     reservation_size = get_num_sites_reserved_per_incidence_at_rank(rank)
#     num_incidence_reservations = get_num_incidence_reservations_at_rank(
#         rank, surface_size
#     )

#     is_reservation_holdover = is_site_reservation_holdover_at_rank(
#         site, surface_size, rank
#     )
#     if is_reservation_holdover:
#         reservation_size //= 2
#         num_incidence_reservations *= 2

#     longevity_ordered_reservation_position = site // reservation_size

#     reservation_index = pylib.longevity_ordering_descending.get_longevity_index_of_mapped_position(
#         longevity_ordered_reservation_position,
#         num_incidence_reservations,
#         polarity=True,
#     )
#     return reservation_index

pylib.jupyter_hide_toggle(hide=True)


In [ ]:
# test get_reservation_index_of_site_at_rank
# **needs to be updated**
# assert [get_reservation_index_of_site_at_rank(site, 16, 0) for site in range(16)] == [
#     0,
#     8,
#     4,
#     9,
#     2,
#     10,
#     5,
#     11,
#     1,
#     12,
#     6,
#     13,
#     3,
#     14,
#     7,
#     15,
# ]

# for rank in 1, 2:
#     assert [get_reservation_index_of_site_at_rank(site, 16, rank) for site in range(16)] == [
#         0,
#         0,
#         4,
#         4,
#         2,
#         2,
#         5,
#         5,
#         1,
#         1,
#         6,
#         6,
#         3,
#         3,
#         7,
#         7,
#     ]

# for rank in range(3, 7):
#     assert [get_reservation_index_of_site_at_rank(site, 16, rank) for site in range(16)] == [
#         0,
#         0,
#         0,
#         4,
#         2,
#         2,
#         2,
#         5,
#         1,
#         1,
#         1,
#         6,
#         3,
#         3,
#         3,
#         7,
#     ]

# for rank in range(7, 14):
#     assert [get_reservation_index_of_site_at_rank(site, 16, rank) for site in range(16)] == [
#         0,
#         0,
#         0,
#         0,
#         2,
#         2,
#         2,
#         2,
#         1,
#         1,
#         1,
#         1,
#         3,
#         3,
#         3,
#         3,
#     ]

# for rank in range(15, 21):
#     assert [get_reservation_index_of_site_at_rank(site, 16, 15) for site in range(16)] == [
#         0,
#         0,
#         0,
#         0,
#         0,
#         2,
#         2,
#         2,
#         1,
#         1,
#         1,
#         1,
#         1,
#         3,
#         3,
#         3,
#     ]

pylib.jupyter_hide_toggle(hide=True)


In [ ]:
# def get_hanoi_incidence_of_site_at_rank(
#     site: int, surface_size: int, rank: int
# ) -> int:
#     # this function deals with incidence wraparound over number of reservations
#     hanoi_value = get_hanoi_value_of_site_at_rank(site, surface_size, rank)
#     incidence_count = (
#         pylib.hanoi.get_incidence_count_of_hanoi_value_through_index(
#             hanoi_value,
#             rank,
#         )
#     )
#     incidence_count = max(incidence_count, 1)
#     max_incidence = incidence_count - 1

#     num_incidence_reservations_naive = get_num_incidence_reservations_at_rank(
#         rank, surface_size
#     )
#     is_site_reservation_holdover_at_rank_ = is_2x_reservation_eligible(
#         hanoi_value,
#         surface_size,
#         rank,
#     )
#     num_incidence_reservations = num_incidence_reservations_naive * (
#         is_site_reservation_holdover_at_rank_ + 1
#     )

#     reservation_index = get_reservation_index_of_site_at_rank(
#         site, surface_size, rank
#     )
#     assert reservation_index < num_incidence_reservations

#     if max_incidence >= num_incidence_reservations:
#         excess_incidence = (
#             max_incidence - reservation_index
#         ) % num_incidence_reservations
#         candidate_incidence = max_incidence - excess_incidence
#         candidate_rank = pylib.hanoi.get_index_of_hanoi_value_nth_incidence(
#             hanoi_value,
#             candidate_incidence,
#         )
#         assert candidate_rank <= rank
#         if candidate_rank == rank:
#             return candidate_incidence
#         else:
#             return get_hanoi_incidence_of_site_at_rank(
#                 site, surface_size, candidate_rank
#             )
#     else:
#         return reservation_index
#

pylib.jupyter_hide_toggle(hide=True)


In [ ]:
# test get_hanoi_incidence_of_site_at_rank
# **needs to be updated**
# assert [get_hanoi_incidence_of_site_at_rank(0, 64, rank) for rank in range(20)] == [
#         # deposition site
#         #     # num_reservations
#         #     #     # hanoi sequence (0-based):
#     0,  # 0,  # 64, # 0,
#     0,  # 1,  # 32, # 1,
#     0,  # 32, # 32, # 0,
#     0,  # 2,  # 16, # 2,
#     0,  # 16, # 16, # 0,
#     0,  # 33, # 16, # 1,
#     0,  # 48, # 16, # 0,
#     0,  # 3,  # 16, # 3,
#     0,  # 8,  # 16, # 0,
#     0,  # 17, # 16, # 1,
#     0,  # 24, # 16, # 0,
#     0,  # 34, # 16, # 2,
#     0,  # 40, # 16, # 0,
#     0,  # 49, # 16, # 1,
#     0,  # 56, # 16, # 0,
#     0,  # 4,  # 8,  # 4,
#     8,  # 0,  # 8,  # 0,
#     8,  # 9,  # 8,  # 1,
#     8,  # 32, # 8,  # 0,
#     8,  # 25, # 8,  # 1,
# ]
# assert [get_hanoi_incidence_of_site_at_rank(1, 64, rank) for rank in range(20)] == [
#         # deposition site
#         #     # num_reservations
#         #     #     # hanoi sequence (0-based):
#     32, # 0,  # 64, # 0,
#     0,  # 1,  # 32, # 1,
#     0,  # 32, # 32, # 0,
#     0,  # 2,  # 16, # 2,
#     0,  # 16, # 16, # 0,
#     0,  # 33, # 16, # 1,
#     0,  # 48, # 16, # 0,
#     0,  # 3,  # 16, # 3,
#     0,  # 8,  # 16, # 0,
#     0,  # 17, # 16, # 1,
#     0,  # 24, # 16, # 0,
#     0,  # 34, # 16, # 2,
#     0,  # 40, # 16, # 0,
#     0,  # 49, # 16, # 1,
#     0,  # 56, # 16, # 0,
#     0,  # 4,  # 8,  # 4,
#     0,  # 0,  # 8,  # 0,
#     0,  # 9,  # 8,  # 1,
#     0,  # 32, # 8,  # 0,
#     0,  # 25, # 8,  # 1,
# ]
# assert [get_hanoi_incidence_of_site_at_rank(32, 64, rank) for rank in range(20)] == [
#         # deposition site
#         #     # num_reservations
#         #     #     # hanoi sequence (0-based):
#     1,  # 0,  # 64, # 0,
#     1,  # 1,  # 32, # 1,
#     1,  # 32, # 32, # 0,
#     1,  # 2,  # 16, # 2,
#     1,  # 16, # 16, # 0,
#     1,  # 33, # 16, # 1,
#     1,  # 48, # 16, # 0,
#     1,  # 3,  # 16, # 3,
#     1,  # 8,  # 16, # 0,
#     1,  # 17, # 16, # 1,
#     1,  # 24, # 16, # 0,
#     1,  # 34, # 16, # 2,
#     1,  # 40, # 16, # 0,
#     1,  # 49, # 16, # 1,
#     1,  # 56, # 16, # 0,
#     1,  # 4,  # 8,  # 4,
#     1,  # 0,  # 8,  # 0,
#     1,  # 9,  # 8,  # 1,
#     9,  # 32, # 8,  # 0,
#     9,  # 25, # 8,  # 1,
# ]
# assert [get_hanoi_incidence_of_site_at_rank(63, 64, rank) for rank in range(20)] == [
#          # deposition site
#          #     # num_reservations
#          #     #     # hanoi sequence (0-based):
#     63,  # 0,  # 64, # 0,
#     31,  # 1,  # 32, # 1,
#     31,  # 32, # 32, # 0,
#     31,  # 2,  # 16, # 2,
#     31,  # 16, # 16, # 0,
#     31,  # 33, # 16, # 1,
#     31,  # 48, # 16, # 0,
#     15,  # 3,  # 16, # 3,
#     15,  # 8,  # 16, # 0,
#     15,  # 17, # 16, # 1,
#     15,  # 24, # 16, # 0,
#     15,  # 34, # 16, # 2,
#     15,  # 40, # 16, # 0,
#     15,  # 49, # 16, # 1,
#     15,  # 56, # 16, # 0,
#     15,  # 4,  # 8,  # 4,
#     15,  # 0,  # 8,  # 0,
#     15,  # 9,  # 8,  # 1,
#     15,  # 32, # 8,  # 0,
#     15,  # 25, # 8,  # 1,
# ]

pylib.jupyter_hide_toggle(hide=True)


In [ ]:
# test get_hanoi_incidence_of_site_at_rank
# **needs to be updated**
# assert [get_hanoi_incidence_of_site_at_rank(0, 16, rank) for rank in range(21)] == [
#         # deposition_site
#         #     # num_reservations
#         #     #     # hanoi sequence (0-based):
#     0,  # 0,  # 16, # 0,
#     0,  # 1,  # 8,  # 1,
#     0,  # 8,  # 8,  # 0,
#     0,  # 2,  # 4,  # 2,
#     0,  # 4,  # 4,  # 0,
#     0,  # 9,  # 4,  # 1,
#     0,  # 12, # 4,  # 0,
#     0,  # 3,  # 4,  # 3,
#     4,  # 0,  # 4,  # 0,
#     4,  # 5,  # 4,  # 1,
#     4,  # 8,  # 4,  # 0,
#     4,  # 10, # 4,  # 2,
#     4,  # 4,  # 4,  # 0,
#     4,  # 13, # 4,  # 1,
#     4,  # 12, # 4,  # 0,
#     4,  # 4,  # 2,  # 4,
#     8,  # 0,  # 2,  # 0,
#     8,  # 1,  # 2,  # 1,
#     8,  # 8,  # 2,  # 0,
#     8,  # 6,  # 2,  # 2,
#     10, # 0,  # 2,  # 0,
# ]
# assert [get_hanoi_incidence_of_site_at_rank(1, 16, rank) for rank in range(21)] == [
#         # deposition_site
#         #     # num_reservations
#         #     #     # hanoi sequence (0-based):
#     8,  # 0,  # 16, # 0,
#     0,  # 1,  # 8,  # 1,
#     0,  # 8,  # 8,  # 0,
#     0,  # 2,  # 4,  # 2,
#     0,  # 4,  # 4,  # 0,
#     0,  # 9,  # 4,  # 1,
#     0,  # 12, # 4,  # 0,
#     0,  # 3,  # 4,  # 3,
#     0,  # 0,  # 4,  # 0,
#     0,  # 5,  # 4,  # 1,
#     0,  # 8,  # 4,  # 0,
#     0,  # 10, # 4,  # 2,
#     0,  # 4,  # 4,  # 0,
#     0,  # 13, # 4,  # 1,
#     0,  # 12, # 4,  # 0,
#     0,  # 4,  # 2,  # 4,
#     0,  # 0,  # 2,  # 0,
#     4,  # 1,  # 2,  # 1,
#     4,  # 8,  # 2,  # 0,
#     4,  # 6,  # 2,  # 2,
#     4,  # 0,  # 2,  # 0,
# ]
# assert [get_hanoi_incidence_of_site_at_rank(2, 16, rank) for rank in range(21)] == [
#         # rank
#         #    # deposition_site
#         #    #     # num_reservations
#         #    #     #     # hanoi sequence (0-based):
#     4,  # 0  # 0,  # 16, # 0,
#     4,  # 1  # 1,  # 8,  # 1,
#     4,  # 2  # 8,  # 8,  # 0,
#     0,  # 3  # 2,  # 4,  # 2,
#     0,  # 4  # 4,  # 4,  # 0,
#     0,  # 5  # 9,  # 4,  # 1,
#     0,  # 6  # 12, # 4,  # 0,
#     0,  # 7  # 3,  # 4,  # 3,
#     0,  # 8  # 0,  # 4,  # 0,
#     0,  # 9  # 5,  # 4,  # 1,
#     0,  # 10 # 8,  # 4,  # 0,
#     0,  # 11 # 10, # 4,  # 2,
#     0,  # 12 # 4,  # 4,  # 0,
#     0,  # 13 # 13, # 4,  # 1,
#     0,  # 14 # 12, # 4,  # 0,
#     0,  # 15 # 4,  # 2,  # 4,
#     0,  # 16 # 0,  # 2,  # 0,
#     0,  # 17 # 1,  # 2,  # 1,
#     0,  # 18 # 8,  # 2,  # 0,
#     0,  # 19 # 6,  # 2,  # 2,
#     0,  # 20 # 0,  # 2,  # 0,
# ]
# assert [get_hanoi_incidence_of_site_at_rank(8, 16, rank) for rank in range(21)] == [
#         # deposition_site
#         #     # num_reservations
#         #     #     # hanoi sequence (0-based):
#     1,  # 0,  # 16, # 0,
#     1,  # 1,  # 8,  # 1,
#     1,  # 8,  # 8,  # 0,
#     1,  # 2,  # 4,  # 2,
#     1,  # 4,  # 4,  # 0,
#     1,  # 9,  # 4,  # 1,
#     1,  # 12, # 4,  # 0,
#     1,  # 3,  # 4,  # 3,
#     1,  # 0,  # 4,  # 0,
#     1,  # 5,  # 4,  # 1,
#     5,  # 8,  # 4,  # 0,
#     5,  # 10, # 4,  # 2,
#     5,  # 4,  # 4,  # 0,
#     5,  # 13, # 4,  # 1,
#     5,  # 12, # 4,  # 0,
#     5,  # 4,  # 2,  # 4,
#     5,  # 0,  # 2,  # 0,
#     5,  # 1,  # 2,  # 1,
#     9,  # 8,  # 2,  # 0,
#     9,  # 6,  # 2,  # 2,
#     9,  # 0,  # 2,  # 0,
# ]
# assert [get_hanoi_incidence_of_site_at_rank(15, 16, rank) for rank in range(21)] == [
#         # deposition_site
#         #     # num_reservations
#         #     #     # hanoi sequence (0-based):
#     15, # 0,  # 16, # 0,
#     7,  # 1,  # 8,  # 1,
#     7,  # 8,  # 8,  # 0,
#     7,  # 2,  # 4,  # 2,
#     7,  # 4,  # 4,  # 0,
#     7,  # 9,  # 4,  # 1,
#     7,  # 12, # 4,  # 0,
#     3,  # 3,  # 4,  # 3,
#     3,  # 0,  # 4,  # 0,
#     3,  # 5,  # 4,  # 1,
#     3,  # 8,  # 4,  # 0,
#     3,  # 10, # 4,  # 2,
#     3,  # 4,  # 4,  # 0,
#     3,  # 13, # 4,  # 1,
#     3,  # 12, # 4,  # 0,
#     3,  # 4,  # 2,  # 4,
#     3,  # 0,  # 2,  # 0,
#     3,  # 1,  # 2,  # 1,
#     3,  # 8,  # 2,  # 0,
#     3,  # 6,  # 2,  # 2,
#     3,  # 0,  # 2,  # 0,
# ]

pylib.jupyter_hide_toggle(hide=True)


In [ ]:
@pylib.enforce_typing
def get_reservation_index_elimination_rank(
    hanoi_value: int,
    reservation_index: int,
    surface_size: int,
) -> typing.Optional[int]:

    first_incidence_rank = pylib.hanoi.get_index_of_hanoi_value_nth_incidence(
        hanoi_value, 0
    )
    max_reservations_provided = get_num_reservations_provided(
        hanoi_value=hanoi_value,
        surface_size=surface_size,
        rank=first_incidence_rank,
    )
    if reservation_index == 0:
        # special case because we implicitly assume
        # always at least one reservation index
        res = get_surface_rank_capacity(surface_size) - 1
        assert res
        return res
    elif reservation_index >= max_reservations_provided:
        return None
    else:

        def predicate(rank: int) -> bool:
            return (
                get_num_reservations_provided(
                    hanoi_value=hanoi_value,
                    surface_size=surface_size,
                    rank=rank + 1,
                )
                <= reservation_index
            )

        res = (
            inch.binary_search(
                predicate,
                first_incidence_rank,
                # upper bound prevents assertion errors from out of bounds queries
                get_surface_rank_capacity(surface_size) - 1,
            )
            + 1
        )
        assert res
        return res


pylib.jupyter_hide_toggle()


In [ ]:
# test get_reservation_index_elimination_rank
# helper function for test
def make_num_reservations_provided_df(surface_size: int) -> pd.DataFrame:
    num_generations = min(
        2**12,
        get_surface_rank_capacity(surface_size) - 1,
    )
    records = []
    max_hanoi_value = pylib.hanoi.get_max_hanoi_value_through_index(
        num_generations - 1
    )
    for hanoi_value in range(max_hanoi_value):
        first_incidence_rank = (
            pylib.hanoi.get_index_of_hanoi_value_nth_incidence(hanoi_value, 0)
        )
        for rank in tqdm(range(first_incidence_rank, num_generations)):
            num_reservations_provided = get_num_reservations_provided(
                hanoi_value=hanoi_value,
                surface_size=surface_size,
                rank=rank,
            )
            records.append(
                {
                    "rank": rank,
                    "hanoi value": hanoi_value,
                    "num reservations provided": num_reservations_provided,
                }
            )

    return pd.DataFrame.from_records(records)


pylib.jupyter_hide_toggle(hide=True)


In [ ]:
# test get_reservation_index_elimination_rank
# demo helper function for test
nrp_df = make_num_reservations_provided_df(32)

display(nrp_df)

sns.lineplot(
    data=nrp_df,
    x="rank",
    y="num reservations provided",
    hue="hanoi value",
)

pylib.jupyter_hide_toggle(hide=True)


In [ ]:
# test get_reservation_index_elimination_rank
# actual test

for surface_size in 16, 32, 64:
    nrp_df = make_num_reservations_provided_df(surface_size)

    for hanoi_value, hv_df in nrp_df.groupby("hanoi value"):
        assert hv_df["num reservations provided"].is_monotonic_decreasing
        max_reservations_provided = hv_df["num reservations provided"].max()
        expected = None
        actual = get_reservation_index_elimination_rank(
            hanoi_value=hanoi_value,
            reservation_index=max_reservations_provided,
            surface_size=surface_size,
        )
        assert expected == actual, {
            "expected": expected,
            "actual": actual,
            "hanoi value": hanoi_value,
            "reservation index": max_reservations_provided,
            "surface size": surface_size,
        }
        max_covered_rank = hv_df["rank"].max()

        for nrp, nrp_df_ in hv_df.groupby("num reservations provided"):
            assert nrp
            reservation_index = nrp - 1
            expected_last = nrp_df_["rank"].max()
            actual_last = (
                get_reservation_index_elimination_rank(
                    hanoi_value=hanoi_value,
                    reservation_index=reservation_index,
                    surface_size=surface_size,
                )
                - 1
            )
            assert actual_last >= 0
            if (
                expected_last < max_covered_rank
            ):  # skip where coverage goes off end of sampled ranks
                assert expected_last == actual_last, {
                    "expected last": expected_last,
                    "actual last": actual_last,
                    "hanoi value": hanoi_value,
                    "reservation index": reservation_index,
                    "surface size": surface_size,
                }


pylib.jupyter_hide_toggle(hide=True)


In [ ]:
def iter_candidate_reservation_sizes(
    site: int,
    rank: int,
) -> typing.Iterator[int]:
    # this is going BACK in time
    naive_reservation_size = get_num_sites_reserved_per_incidence_at_rank(rank)

    reservation_size = naive_reservation_size
    while reservation_size:
        candidate_hanoi_value = site % reservation_size
        yield reservation_size
        reservation_size //= 2


In [ ]:
def iter_candidate_hanoi_occupants(
    site: int,
    rank: int,
) -> typing.Iterator[int]:
    # this is going BACK in time
    for candidate_reservation_size in iter_candidate_reservation_sizes(site, rank):
        candidate_hanoi_value = site % candidate_reservation_size
        yield candidate_hanoi_value


pylib.jupyter_hide_toggle()


In [ ]:
# test iter_candidate_hanoi_occupants
for surface_size in 16, 32, 64:
    capacity = get_surface_rank_capacity(surface_size)
    for rank in it.chain(
        range(300),
        map(int, np.linspace(0, capacity, num=100, dtype=int)),
    ):
        {*iter_candidate_hanoi_occupants(0, rank)} == {0}

        for site in range(surface_size):
            assert hstrat_aux.is_nonincreasing(
                iter_candidate_hanoi_occupants(site, rank)
            )

assert {*iter_candidate_hanoi_occupants(1, 2)} == {0, 1}

assert {*iter_candidate_hanoi_occupants(1, 5559)} == {0, 1}

assert {*iter_candidate_hanoi_occupants(6, 5)} == {0, 2}

assert {*iter_candidate_hanoi_occupants(5, 5)} == {0, 1}

assert {*iter_candidate_hanoi_occupants(7, 5)} == {0, 1, 3}


assert {*iter_candidate_hanoi_occupants(6, 16)} == {0, 2, 6}

assert {*iter_candidate_hanoi_occupants(5, 16)} == {0, 1, 5}

assert {*iter_candidate_hanoi_occupants(7, 16)} == {0, 1, 3, 7}

assert {*iter_candidate_hanoi_occupants(8, 16)} == {0}

assert {*iter_candidate_hanoi_occupants(9, 16)} == {0, 1}

pylib.jupyter_hide_toggle(hide=True)


In [ ]:
def iter_candidate_reservation_indices(
    site: int,
    surface_size: int,
    rank: int,
) -> typing.Iterator[int]:
    # this is going BACK in time

    for candidate_reservation_size in iter_candidate_reservation_sizes(site, rank):
        candidate_reservation_position = site // candidate_reservation_size
        candidate_reservation_index = (
            pylib.longevity_ordering_descending.get_longevity_index_of_mapped_position(
                candidate_reservation_position,
                surface_size // candidate_reservation_size,
            )
        )
        yield candidate_reservation_index


In [ ]:
@pylib.log_args_and_result(logger, logging.DEBUG)
def calc_instance_of_deposited_hanoi_value(
    hanoi_value: int,
    reservation_index: int,
    surface_size: int,
    focal_rank: int,
    _is_recurse = False,
) -> int:
    num_reservations_provided = get_num_reservations_provided(
        hanoi_value=hanoi_value,
        surface_size=surface_size,
        rank=focal_rank,
    )
    assert num_reservations_provided

    hanoi_offset = pylib.hanoi.get_hanoi_value_index_offset(hanoi_value)
    hanoi_cadence = pylib.hanoi.get_hanoi_value_index_cadence(hanoi_value)
    hanoi_count = pylib.hanoi.get_incidence_count_of_hanoi_value_through_index(
        hanoi_value,
        focal_rank,
    )
    assert hanoi_count

    # figure out where the modulus lines up at the zero position
    reference_rank = None
    if hanoi_count <= num_reservations_provided:
        reference_rank = hanoi_offset
    else:
        prev_reservations_provided = num_reservations_provided + 1
        # ^ note x2 diff is also covered by this

        prev_reservation_index_elimination_rank = get_reservation_index_elimination_rank(
            hanoi_value,
            prev_reservations_provided - 1,  # subtract 1 to convert count to index
            surface_size,
        )
        if prev_reservation_index_elimination_rank is not None:
            assert prev_reservation_index_elimination_rank >= hanoi_offset

            #  zero alignment occurs one instance before elimination (?)
            if prev_reservation_index_elimination_rank >= hanoi_cadence:
                prev_reservation_index_elimination_rank -= hanoi_cadence
            assert prev_reservation_index_elimination_rank >= 0

        reference_rank = opyt.or_value(
            prev_reservation_index_elimination_rank,
            hanoi_offset,
        )

        # round down to next hanoi instance rank (should this be up?)
        reference_rank = reference_rank - (reference_rank - hanoi_offset) % hanoi_cadence

        logger.debug(
            {
                "hanoi_cadence": hanoi_cadence,
                "hanoi_count": hanoi_count,
                "hanoi_offset": hanoi_offset,
                "hanoi_value": hanoi_value,
                "num_reservations_provided": num_reservations_provided,
                "prev_reservations_provided": prev_reservations_provided,
                "prev_reservation_index_elimination_rank": prev_reservation_index_elimination_rank,
                "reference_rank": reference_rank,
                "surface_size": surface_size,
            },
        )

    assert reference_rank >= 0

    assert reference_rank <= focal_rank
    assert (reference_rank - hanoi_offset) % hanoi_cadence == 0, {
        "hanoi_cadence": hanoi_cadence,
        "hanoi_count": hanoi_count,
        "hanoi_offest": hanoi_offset,
        "hanoi_value": hanoi_value,
        "num_reservations_provided": num_reservations_provided,
        "prev_reservation_index_elimination_rank": prev_reservation_index_elimination_rank,
        "reference_rank": reference_rank,
        "reservation_index": reservation_index,
        "surface_size": surface_size,
        "focal_rank": focal_rank,
    }

    rank_duration = focal_rank - reference_rank
    instance_duration = rank_duration // hanoi_cadence

    logger.debug(
        {
            "focal_rank": focal_rank,
            "hanoi_cadence": hanoi_cadence,
            "hanoi_count": hanoi_count,
            "hanoi_offset": hanoi_offset,
            "hanoi_value": hanoi_value,
            "instance_duration": instance_duration,
            "num_reservations_provided": num_reservations_provided,
            "rank_duration": rank_duration,
            "reference_rank": reference_rank,
            "reservation_index": reservation_index,
            "surface_size": surface_size,
        },
    )

    if instance_duration >= reservation_index:
        instance_offset = reservation_index

        base_instance = instance_duration - (instance_duration % num_reservations_provided)
        if base_instance + instance_offset > instance_duration:
            # need to get a base instance with a valid instance offset above it
            # roll back to the previous base instance if instance offset does not fit
            assert base_instance >= num_reservations_provided
            base_instance -= num_reservations_provided

        assert pylib.hanoi.get_hanoi_value_at_index(reference_rank) == hanoi_value
        reference_instance = pylib.hanoi.get_hanoi_value_incidence_at_index(reference_rank)

        logging.debug(
            {
                "base_instance": base_instance,
                "instance_offset": instance_offset,
                "reference_instance": reference_instance,
            },
        )
        return (
            reference_instance
            + base_instance
            + instance_offset
        )

    else:
        # should recurse at most once
        assert not _is_recurse
        assert reference_rank
        return calc_instance_of_deposited_hanoi_value(
            hanoi_value,
            reservation_index,
            surface_size,
            reference_rank - 1,
            _is_recurse=True,
        )


In [ ]:
@pylib.log_args_and_result(logger, logging.DEBUG)
def calc_rank_of_deposited_hanoi_value(
    hanoi_value: int,
    reservation_index: int,
    surface_size: int,
    focal_rank: int,
) -> int:
    instance = calc_instance_of_deposited_hanoi_value(
        hanoi_value,
        reservation_index,
        surface_size,
        focal_rank,
    )
    res = pylib.hanoi.get_index_of_hanoi_value_nth_incidence(
        hanoi_value,
        instance,
    )
    assert res <= focal_rank, {
        "hanoi_value" : hanoi_value,
        "instance" : instance,
        "focal_rank" : focal_rank,
        "res": res,
        "reservation_index" : reservation_index,
        "surface_size" : surface_size,
    }
    return res


In [ ]:
def get_deposition_rank_at_site(
    site: int, surface_size: int, num_depositions: int
) -> int:

    if num_depositions == 0:
        return 0

    rank = num_depositions - 1

    for candidate_hanoi_value, candidate_reservation_index in zip(
        iter_candidate_hanoi_occupants(site, rank),
        iter_candidate_reservation_indices(site, surface_size, rank),
    ):
        logger.debug(f"{candidate_hanoi_value=}")
        logger.debug(f"{candidate_reservation_index=}")
        deadline_rank = get_reservation_index_elimination_rank(
            candidate_hanoi_value,
            candidate_reservation_index,
            surface_size,
        )
        logger.debug(f"{deadline_rank=}")
        if deadline_rank is None:
            continue  # could this be a break?

        assert deadline_rank, {
            "candidate_hanoi_value": candidate_hanoi_value,
            "candidate_reservation_index": candidate_reservation_index,
            "rank": rank,
            "site": site,
            "surface_size": surface_size,
        }

        focal_rank = min(rank, deadline_rank - 1)
        assert focal_rank <= rank
        logger.debug(f"{focal_rank=}")

        hanoi_count = pylib.hanoi.get_incidence_count_of_hanoi_value_through_index(
            candidate_hanoi_value,
            focal_rank,
        )
        if hanoi_count > candidate_reservation_index:
            res = calc_rank_of_deposited_hanoi_value(
                candidate_hanoi_value,
                candidate_reservation_index,
                surface_size,
                focal_rank,
            )
            assert res <= rank, {
                "candidate_hanoi_value": candidate_hanoi_value,
                "candidate_reservation_index": candidate_reservation_index,
                "deadline_rank": deadline_rank,
                "focal_rank": focal_rank,
                "hanoi_count": hanoi_count,
                "rank": rank,
                "res": res,
                "site": site,
                "surface_size": surface_size,
            }
            return res

    return 0


## Section 3b: Define `get_deposition_rank_at_site`


In [ ]:
# def get_deposition_rank_at_site(
#     site: int, surface_size: int, num_depositions: int
# ) -> int:
#     if num_depositions == 0:
#         return 0

#     rank = num_depositions - 1

#     hanoi_value = get_hanoi_value_of_site_at_rank(site, surface_size, rank)
#     hanoi_incidence = get_hanoi_incidence_of_site_at_rank(site, surface_size, rank)
#     candidate_rank = pylib.hanoi.get_index_of_hanoi_value_nth_incidence(
#         hanoi_value,
#         hanoi_incidence,
#     )

#     if candidate_rank < num_depositions:
#         return candidate_rank
#     else:
#         # haven't got this far in the reservation
#         # what was left here the last time around
#         # todo this recursion is slow and inefficient
#         return get_deposition_rank_at_site(site, surface_size, num_depositions - 1)

pylib.jupyter_hide_toggle()


## Section 4: Evaluate `get_deposition_rank_at_site`


## Section 4a: Unit Test


In [ ]:
# test get_deposition_rank_at_site
# assert [
#     get_deposition_rank_at_site(0, 16, num_depositions) for num_depositions in range(22)
# ] == [
#         # deposition site
#         #     # num_reservations
#         #     #     # hanoi sequence (0-based):
#     0,  # ~~~~ n/a ~~~~
#     0,  # 0,  # 16, # 0,
#     0,  # 1,  # 8,  # 1,
#     0,  # 8,  # 8,  # 0,
#     0,  # 2,  # 4,  # 2,
#     0,  # 4,  # 4,  # 0,
#     0,  # 9,  # 4,  # 1,
#     0,  # 12, # 4,  # 0,
#     0,  # 3,  # 4,  # 3,
#     8,  # 0,  # 4,  # 0,
#     8,  # 5,  # 4,  # 1,
#     8,  # 8,  # 4,  # 0,
#     8,  # 10, # 4,  # 2,
#     8,  # 4,  # 4,  # 0,
#     8,  # 13, # 4,  # 1,
#     8,  # 12, # 4,  # 0,
#     8,  # 4,  # 2,  # 4,
#     16, # 0,  # 2,  # 0,
#     16, # 1,  # 2,  # 1,
#     16, # 8,  # 2,  # 0,
#     16, # 6,  # 2,  # 2,
#     20, # 0,  # 2,  # 0,
# ]
# assert [
#     get_deposition_rank_at_site(1, 16, num_depositions) for num_depositions in range(22)
# ] == [
#         # deposition site
#         #     # num_reservations
#         #     #     # hanoi sequence (0-based):
#     0,  # ~~~~ n/a ~~~~
#     0,  # 0,  # 16, # 0,
#     1,  # 1,  # 8,  # 1,
#     1,  # 8,  # 8,  # 0,
#     1,  # 2,  # 4,  # 2,
#     1,  # 4,  # 4,  # 0,
#     1,  # 9,  # 4,  # 1,
#     1,  # 12, # 4,  # 0,
#     1,  # 3,  # 4,  # 3,
#     1,  # 0,  # 4,  # 0,
#     1,  # 5,  # 4,  # 1,
#     1,  # 8,  # 4,  # 0,
#     1,  # 10, # 4,  # 2,
#     1,  # 4,  # 4,  # 0,
#     1,  # 13, # 4,  # 1,
#     1,  # 12, # 4,  # 0,
#     1,  # 4,  # 2,  # 4,
#     1,  # 0,  # 2,  # 0,
#     17, # 1,  # 2,  # 1,
#     17, # 8,  # 2,  # 0,
#     17, # 6,  # 2,  # 2,
#     17, # 0,  # 2,  # 0,
# ]
# assert [
#     get_deposition_rank_at_site(2, 16, num_depositions) for num_depositions in range(22)
# ] == [
#         # deposition site
#         #     # num_reservations
#         #     #     # hanoi sequence (0-based):
#     0,  # ~~~~ n/a ~~~~
#     0,  # 0,  # 16, # 0,
#     0,  # 1,  # 8,  # 1,
#     0,  # 8,  # 8,  # 0,
#     3,  # 2,  # 4,  # 2,
#     3,  # 4,  # 4,  # 0,
#     3,  # 9,  # 4,  # 1,
#     3,  # 12, # 4,  # 0,
#     3,  # 3,  # 4,  # 3,
#     3,  # 0,  # 4,  # 0,
#     3,  # 5,  # 4,  # 1,
#     3,  # 8,  # 4,  # 0,
#     3,  # 10, # 4,  # 2,
#     3,  # 4,  # 4,  # 0,
#     3,  # 13, # 4,  # 1,
#     3,  # 12, # 4,  # 0,
#     3,  # 4,  # 2,  # 4,
#     3,  # 0,  # 2,  # 0,
#     3,  # 1,  # 2,  # 1,
#     3,  # 8,  # 2,  # 0,
#     3,  # 6,  # 2,  # 2,
#     3,  # 0,  # 2,  # 0,
# ]
# assert [
#     get_deposition_rank_at_site(15, 16, num_depositions)
#     for num_depositions in range(22)
# ] == [
#         # deposition site
#         #     # num_reservations
#         #     #     # hanoi sequence (0-based):
#     0,  # ~~~~ n/a ~~~~
#     0,  # 0,  # 16, # 0,
#     0,  # 1,  # 8,  # 1,
#     0,  # 8,  # 8,  # 0,
#     0,  # 2,  # 4,  # 2,
#     0,  # 4,  # 4,  # 0,
#     0,  # 9,  # 4,  # 1,
#     0,  # 12, # 4,  # 0,
#     0,  # 3,  # 4,  # 3,
#     0,  # 0,  # 4,  # 0,
#     0,  # 5,  # 4,  # 1,
#     0,  # 8,  # 4,  # 0,
#     0,  # 10, # 4,  # 2,
#     0,  # 4,  # 4,  # 0,
#     0,  # 13, # 4,  # 1,
#     0,  # 12, # 4,  # 0,
#     0,  # 4,  # 2,  # 4,
#     0,  # 0,  # 2,  # 0,
#     0,  # 1,  # 2,  # 1,
#     0,  # 8,  # 2,  # 0,
#     0,  # 6,  # 2,  # 2,
#     0,  # 0,  # 2,  # 0,
# ]
# assert [
#     get_deposition_rank_at_site(8, 16, num_depositions) for num_depositions in range(22)
# ] == [
#         # deposition site
#         #     # num_reservations
#         #     #     # hanoi sequence (0-based):
#     0,  # ~~~~ n/a ~~~~
#     0,  # 0,  # 16, # 0,
#     0,  # 1,  # 8,  # 1,
#     2,  # 8,  # 8,  # 0,
#     2,  # 2,  # 4,  # 2,
#     2,  # 4,  # 4,  # 0,
#     2,  # 9,  # 4,  # 1,
#     2,  # 12, # 4,  # 0,
#     2,  # 3,  # 4,  # 3,
#     2,  # 0,  # 4,  # 0,
#     2,  # 5,  # 4,  # 1,
#     10, # 8,  # 4,  # 0,
#     10, # 10, # 4,  # 2,
#     10, # 4,  # 4,  # 0,
#     10, # 13, # 4,  # 1,
#     10, # 12, # 4,  # 0,
#     10, # 4,  # 2,  # 4,
#     10, # 0,  # 2,  # 0,
#     10, # 1,  # 2,  # 1,
#     18, # 8,  # 2,  # 0,
#     18, # 6,  # 2,  # 2,
#     18, # 0,  # 2,  # 0,
# ]
# assert [
#     get_deposition_rank_at_site(9, 16, num_depositions) for num_depositions in range(23)
# ] == [
#         # deposition site
#         #     # num_reservations
#         #     #     # hanoi sequence (0-based):
#     0,  # ~~~~ n/a ~~~~
#     0,  # 0,  # 16, # 0,
#     0,  # 1,  # 8,  # 1,
#     0,  # 8,  # 8,  # 0,
#     0,  # 2,  # 4,  # 2,
#     0,  # 4,  # 4,  # 0,
#     5,  # 9,  # 4,  # 1,
#     5,  # 12, # 4,  # 0,
#     5,  # 3,  # 4,  # 3,
#     5,  # 0,  # 4,  # 0,
#     5,  # 5,  # 4,  # 1,
#     5,  # 8,  # 4,  # 0,
#     5,  # 10, # 4,  # 2,
#     5,  # 4,  # 4,  # 0,
#     5,  # 13, # 4,  # 1,
#     5,  # 12, # 4,  # 0,
#     5,  # 4,  # 2,  # 4,
#     5,  # 0,  # 2,  # 0,
#     5,  # 1,  # 2,  # 1,
#     5,  # 8,  # 2,  # 0,
#     5,  # 6,  # 2,  # 2,
#     5,  # 0,  # 2,  # 0,
#     21, # 9,  # 2,  # 1,
# ]

pylib.jupyter_hide_toggle(hide=True)


## Section 4b: Integration Test


In [ ]:
for surface_size_exp in range(2, 12):
    logger.setLevel(logging.WARNING)
    surface_size = 2**surface_size_exp
    num_generations = min(
        2**18,
        get_surface_rank_capacity(surface_size) - 1,
    )
    print(f"evaluating surface size {surface_size} for {num_generations} generations")

    surface_deposition_ranks = [0] * surface_size
    for rank in range(num_generations):
        target_site = get_deposition_site_at_rank(rank, surface_size)
        print(
            "rank", rank,
            "hanoi", pylib.hanoi.get_hanoi_value_at_index(rank),
            "target_site", target_site
        )
        surface_deposition_ranks[target_site] = rank

        # TODO fixme
        for site, actual_deposition_rank in enumerate(surface_deposition_ranks):

            try:
                calculated_deposition_rank = get_deposition_rank_at_site(
                    site,
                    surface_size,
                    rank + 1,
                )
            except AssertionError as e:
                logger.warning(f"AssertionError {e}")
                logger.warning({
                    "actual_deposition_rank": actual_deposition_rank,
                    "rank": rank,
                    "site": site,
                })
                calculated_deposition_rank = None

            if calculated_deposition_rank != actual_deposition_rank:

                logger.setLevel(logging.DEBUG)
                get_deposition_rank_at_site(
                    site=site,
                    surface_size=surface_size,
                    num_depositions=rank + 1,
                )
                logger.setLevel(logging.WARNING)

                assert False, {
                    "actual deposition rank": actual_deposition_rank,
                    "calculated deposition rank": calculated_deposition_rank,
                    "hanoi value": pylib.hanoi.get_hanoi_value_at_index(rank),
                    "num depositions": rank + 1,
                    "rank": rank,
                    "site": site,
                }

pylib.jupyter_hide_toggle()
